# Merge Datasets

In [1]:
# Data processing
import numpy as np
import pandas as pd
import requests
import math
import csv
import re # Regular Expressions
from datetime import datetime

# Geographies
import shapely
from shapely.geometry import mapping, shape
from shapely.geometry import Point, LineString, Polygon
from descartes import PolygonPatch

# Common DGLIM utilities
import dglim
from dglim import City
dglim.setProjectPath('../')

%matplotlib inline

## Collect Active Business Data

In [2]:
# Load datasets
act_bus_df = pd.read_excel(dglim.datasets_path + 'DGLIM Survey Data/city of Gainesville active businesses dataset with uniqueid (fixed coords).xlsx')
naics_df = pd.read_excel(dglim.datasets_path + 'NAICS_BusinessTypes.xlsx')

# Clean them up
business_slice_df = act_bus_df[[
    'uniqueid',
    'ID',
    'Name',
    'Business Type',
    'Start Date',
]].copy() \
.rename(columns={
    'uniqueid' : 'DGLIM ID',
    'ID' : 'GRU ID'
}) \
.dropna(subset=['DGLIM ID'])

business_slice_df['DGLIM ID'] = business_slice_df['DGLIM ID'].astype('int')
business_slice_df.set_index('DGLIM ID', drop=False, inplace=True)

# Fix a bad start date
if (str(business_slice_df['Start Date'][6756]) == '2106-09-24 00:00:00'):
    fix_year = business_slice_df['Start Date'][6756].replace(year=2016);
    business_slice_df.set_value(index=6756, col='Start Date', value=fix_year)

# Add NAICS types
type_to_naics_map = pd.Series(
    naics_df['NAICS Business Type'].values,
    index=naics_df['Business types'])
business_slice_df['NAICS Type'] = business_slice_df['Business Type'].map(type_to_naics_map)

business_slice_df[:1]

,DGLIM ID,GRU ID,Name,Business Type,Start Date,NAICS Type
DGLIM ID,,,,,,
1,1,21546,"WESTIN, JEAN",ATTORNEY/LAWYER,1989-10-02,"Professional, Scientific, and Technical Services"


## Collect Survey Data

In [27]:
len(survey_df)

751

In [22]:
# Load dataset
survey_df = pd.read_excel(dglim.datasets_path + 'DGLIM Survey Data/DGLIM_data_final_111417.xlsx')

# Clean it up
survey_slice_df = survey_df.copy() \
.drop(axis='index', labels=survey_df.index[:2]) \
.drop(axis='columns', labels=[
    'Duration (in seconds)',
    'Email',
    'EndDate',
    'Location',
    'Progress',
    'RecordedDate',
    'StartDate',
    'Unnamed: 88', # Empty column
]).rename(columns={
    'uniqueid' : 'DGLIM ID',
    'Finished' : 'Finished Survey',

    'Q7'  : 'Number of Employees',
    'Q43' : 'Successfulness',
    'Q47' : 'Revenue in 2016',
    'Q48' : 'Expenses in 2016',
    'Q49' : 'Investment in 2016',
    'Q50' : 'Revenue in 2017',
    'Q51' : 'Expenses in 2017',
    'Q52' : 'Investment in 2017',
    
    'Q29_8' : 'Q29_7'
}) \
.dropna(subset=['DGLIM ID'])

survey_slice_df['DGLIM ID'] = survey_slice_df['DGLIM ID'].astype('int')
survey_slice_df.set_index('DGLIM ID', drop=False, inplace=True)

# Label successfulness
success_dict = {
    1 : 'Very Successful',
    2 : 'Somewhat Successful',
    3 : 'Somewhat Unsuccessful',
    4 : 'Very Unsuccessful'
}
survey_slice_df['Successfulness'] = survey_slice_df['Successfulness'].map(success_dict)

# Sort alphabetically
columns = sorted(survey_slice_df.columns)

# Move unnamed columns to the end
useful_columns = [x for x in columns if x[0] != 'Q']
unused_columns = [x for x in columns if x[0] == 'Q']

columns = useful_columns + unused_columns
survey_slice_df = survey_slice_df.reindex(columns=columns)

survey_slice_df.head()

,DGLIM ID,Expenses in 2016,Expenses in 2017,Finished Survey,Investment in 2016,Investment in 2017,Number of Employees,Revenue in 2016,Revenue in 2017,Successfulness,...,Q44_2,Q44_3,Q44_4,Q44_5,Q44_5_TEXT,Q45,Q46,Q5,Q8,Q9
DGLIM ID,,,,,,,,,,,,,,,,,,,,,
16,16,18000,7500,1,0,900,0,23000,12000,NaN,...,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,0,NaN
17,17,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Somewhat Successful,...,1,NaN,NaN,NaN,NaN,6,NaN,NaN,0,NaN
24,24,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,25,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Somewhat Successful,...,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,0,NaN
38,38,NaN,NaN,0,NaN,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN


## Collect Location Data

In [4]:
locations_slice_df = dglim.loadData("Active Business Locations")
locations_slice_df[:1]

,DGLIM ID,Address,Latitude,Longitude
DGLIM ID,,,,
1,1,1215 NW 12TH AVE,29.663009,-82.337822


## Collect RTS Data

In [5]:
rts_slice_df = dglim.loadData('Bus Stop Distances')
rts_slice_df[:1]

,DGLIM ID,Distance to Bus Stop
DGLIM ID,,
16,16,110.521226


## Collect Nearby Crimes Data

In [6]:
# Prepend "Nearby Crimes in " to each column
crimes_slice_df = dglim.loadData('Crimes Within 500m by GRU ID')
crimes_slice_df.rename(inplace=True,
    columns=dict(zip(
        crimes_slice_df.columns, 
        ['Nearby Crimes in ' + str(x) for x in crimes_slice_df.columns]
    ))
)

# Add 'DGLIM ID' as a column for easier merging
crimes_slice_df['DGLIM ID'] = crimes_slice_df.index
crimes_slice_df[:1]

,Nearby Crimes in 2011,Nearby Crimes in 2012,Nearby Crimes in 2013,Nearby Crimes in 2014,Nearby Crimes in 2015,Nearby Crimes in 2016,Nearby Crimes in 2017,DGLIM ID
DGLIM ID,,,,,,,,
16,79,84,79,65,32,48,64,16


## Collect Parcel Data

In [7]:
parcels_slice_df = dglim.loadData('Active Business Parcels').reset_index().set_index('DGLIM ID', drop=False)
parcels_slice_df[:1]

,DGLIM ID,Parcel
DGLIM ID,,
1,1,09494000000


## Collect Electricity Consumption

In [8]:
# Get per-parcel electricity consumption
parcel_consumption_df = dglim.loadData('Parcel Electricity Consumption').reset_index()

# Map consumption to individual businesses by parcel
electricity_slice_df = parcels_slice_df.merge(parcel_consumption_df, on=['Parcel'], how='left')[[
    'DGLIM ID',
    'Percent Change in KWH'
]].set_index('DGLIM ID', drop=False)
electricity_slice_df[:1]

,DGLIM ID,Percent Change in KWH
DGLIM ID,,
1,1,-0.075


## Collect Permits, Zoning Violations, and Building Code Violations Data

In [11]:
# Get per-parcel counts
parcel_permit_violation_counts_df = dglim.loadData('Parcel Permit and Violation Counts')

# Map parcel counts to individual businesses by parcel
permit_violation_counts_slice_df = parcels_slice_df.merge(
    parcel_permit_violation_counts_df, on=['Parcel'], how='left') \
    .set_index('DGLIM ID', drop=False).drop(axis='columns', labels='Parcel').fillna(0)
permit_violation_counts_slice_df[:1]

,DGLIM ID,Number of Permits,Number of Zoning Violations,Number of Building Code Violations
DGLIM ID,,,,
1,1,0.0,0.0,0.0


## Collect Census Tract/Block Data

In [12]:
# Load dataset
business_blocks_df = dglim.loadData('Block Results')

# Clean it up
tract_slice_df = business_blocks_df.dropna().set_index('DGLIM ID')[[
    'Tract',
    'Block Group'
]]
tract_slice_df[['Tract', 'Block']] = tract_slice_df[['Tract', 'Block Group']].astype('int')

tract_slice_df[:1]

,Tract,Block Group,Block
DGLIM ID,,,
6873,1603,1,1


## Build a Master Dataset

In [13]:
master_df = pd.DataFrame(columns=['DGLIM ID'])

# Add business data
master_df = master_df.merge(business_slice_df, on=['DGLIM ID'], how='right')

# Add location data
master_df = master_df.merge(locations_slice_df, on=['DGLIM ID'], how='left')

# Add parcel data
master_df.merge(parcels_slice_df, on=['DGLIM ID'], how='left')

# Add tract/block data
master_df[['Tract', 'Block']] = tract_slice_df[['Tract', 'Block']].astype('str')
master_df['Tract-Block'] = master_df['Tract'] + '-' + master_df['Block']

# Add bus stop distances data
master_df = master_df.merge(rts_slice_df, on=['DGLIM ID'], how='left')

# Add nearby crimes data
master_df = master_df.merge(crimes_slice_df, on=['DGLIM ID'], how='left')

# Add electricity consumption data
master_df = master_df.merge(electricity_slice_df, on=['DGLIM ID'], how='left')

# Add permits, zoning violations, and building code violations data
master_df = master_df.merge(permit_violation_counts_slice_df, on=['DGLIM ID'], how='left')

# Add survey data
master_df = master_df.merge(survey_slice_df, on=['DGLIM ID'], how='left')

master_df.set_index('DGLIM ID', drop=False, inplace=True)
master_df.head()

,DGLIM ID,GRU ID,Name,Business Type,Start Date,NAICS Type,Address,Latitude,Longitude,Tract,...,Q44_2,Q44_3,Q44_4,Q44_5,Q44_5_TEXT,Q45,Q46,Q5,Q8,Q9
DGLIM ID,,,,,,,,,,,,,,,,,,,,,
1,1,21546,"WESTIN, JEAN",ATTORNEY/LAWYER,1989-10-02,"Professional, Scientific, and Technical Services",1215 NW 12TH AVE,29.663009,-82.337822,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,29778,JERRY BUSH AUTO REPAIR,AUTO REPAIR GARAGE,2001-04-01,"Professional, Scientific, and Technical Services",1311 NW 5TH AVE,29.655734,-82.339970,301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,36594,"FLORIDA BELLS, LLC",RESTAURANT,2010-03-24,Accommodation and Food Services,2224 NW 13TH ST,29.673707,-82.339143,1000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,12906,"DARWIN, HOLLY",MASSAGE THERAPIST,1995-02-01,Health Care and Social Assistance,"4101 NW 37TH PL, SUITE B",29.687533,-82.387161,1100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,42390,CONRAD YELVINGTON DISTRIBUTORS,RETAIL MERCHANT,2015-11-15,Retail Trade,7605 NW 13TH ST,29.726447,-82.370726,1100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*** Save master dataset ***

In [14]:
dglim.saveData(master_df, 'Master Dataset')

# Make Tract/Block Datasets

*** Load raw data ***

In [15]:
ibb_data_df = dglim.loadData('IBB Data')

ibb_data_df[:1].columns

Index([u'Tract', u'Block', u'Population Working', u'Average Income',
       u'Tract-Block', u'HD01_VD01', u'HD01_VD02', u'HD01_VD03', u'HD01_VD04',
       u'HD01_VD05', u'HD01_VD06', u'HD01_VD07', u'HD01_VD08', u'HD01_VD09',
       u'HD01_VD10', u'HD01_VD11', u'HD01_VD12', u'HD01_VD13', u'HD01_VD14',
       u'HD01_VD15', u'HD01_VD16', u'HD01_VD17'],
      dtype='object')

## Build Blocks Dataset

In [16]:
# Build using basic tract data
blocks_df = pd.DataFrame(ibb_data_df[['Tract', 'Block', 'Tract-Block', 'Population Working', 'Average Income']])
blocks_df['Tract'] = blocks_df['Tract'].dropna().astype('int').astype('str')
blocks_df['Block'] = blocks_df['Block'].dropna().astype('int').astype('str')
blocks_df['Population Working'] = blocks_df['Population Working'].fillna(0).astype('int')

blocks_df.set_index('Tract-Block', inplace=True)
blocks_df.dropna(axis='index', inplace=True)
blocks_df.drop_duplicates(inplace=True)
blocks_df.sort_index(inplace=True)

# Count businesses per block
merged_locations_df = master_df.drop_duplicates(subset=['GRU ID', 'Address'])
blocks_df['Total Businesses'] = merged_locations_df['Tract-Block'].value_counts()

# Count success per block
block_success_counts_df = master_df.set_index('Tract-Block', drop=False)['Tract-Block'].drop_duplicates().apply(lambda x:
    master_df[master_df['Tract-Block'] == x]['Successfulness'].value_counts()).fillna(0).astype('int')
blocks_df = blocks_df.join(block_success_counts_df)

# Replace NaNs with zeros
# blocks_df = blocks_df.fillna(0).astype('int')

dglim.saveData(blocks_df, 'Blocks Dataset')
blocks_df[:5]

,Tract,Block,Population Working,Average Income,Total Businesses,Somewhat Successful,Somewhat Unsuccessful,Very Successful,Very Unsuccessful
Tract-Block,,,,,,,,,
1000-1,1000,1,336,109985,34.0,0.0,0.0,0.0,0.0
1000-2,1000,2,428,69147,59.0,3.0,0.0,4.0,1.0
1000-3,1000,3,773,28835,79.0,1.0,1.0,4.0,0.0
1000-4,1000,4,582,66056,18.0,3.0,0.0,0.0,0.0
1000-5,1000,5,278,133075,7.0,0.0,0.0,1.0,0.0


## Build Tracts Dataset

*** Load data ***

In [17]:
pbt_df = dglim.loadData('PBT Data') # Population by Tract

pbt_df[:1]

,Tract,Population,Population Female,Population Over 18,Population Female Over 18,GEOID,HC01_VC08,HC01_VC09,HC01_VC10,HC01_VC11,...,HC01_VC95,HC01_VC96,HC01_VC97,HC01_VC98,HC01_VC99,HC01_VC100,HC01_VC101,HC01_VC102,HC01_VC104,HC01_VC109
Tract,,,,,,,,,,,,,,,,,,,,,
200,200,7295,3839,6779,3553,12001000200,26,0,30,1352,...,1054,0,181,97,0,122,0,122,3175,3226


*** Compile data ***

** NOTE: ** for tracts that are only partially within Gainesville's city limits, the listed population is scaled by the fraction of the tract's land area that lies within those city limits. This assumes that population density is mostly uniform across individual tracts; this causes some issue with population counts, especially in the southeast region.

In [18]:
tracts_df = pbt_df.copy()

# Merge datasets
tracts_df = tracts_df.join(blocks_df.groupby('Tract').sum(), how='outer').sort_index()

# Drop unidentified columns
drop_columns = [x for x in tracts_df.columns if x[0:4] == 'HC01']
tracts_df.drop(labels=drop_columns, axis='columns', inplace=True)

# Replace NaNs with zeros
tracts_df = tracts_df.fillna(0) # .astype('int') # Long values don't like converting to 'int'

# Count number of respondents per tract
tracts_df['Total Responses'] = \
    tracts_df[[
        'Somewhat Successful',
        'Somewhat Unsuccessful',
        'Very Successful',
        'Very Unsuccessful'
    ]].sum(axis='columns')

*** Count number of businesses in each tract ***

In [19]:
# Using block counts, tracts contained 4,664 total businesses (Down from ~5,500; ~500 don't have fixed locations
#  in Gainesville; ~500 locations could not be matched by the Census geocoder)
gainesville_shape = dglim.getGainesvilleShape()
tract_shapes = dglim.getTractShapes(gainesville_shape)

def shapeContainsRow(this_shape, row):
    point = shapely.geometry.Point(row['Longitude'], row['Latitude'])
    return this_shape.contains(point)

merged_locations_df = master_df.drop_duplicates(subset=['GRU ID', 'Address'])

# Let's count number of businesses using GPS coordinates
outside_list = list()
tracts_df['Total Businesses'] = 0
for i, tract in tracts_df.iterrows():
    if i not in tract_shapes:
        outside_list.append(i)
        continue

    tract_shape_meta = tract_shapes[i]
    tract_shape = tract_shape_meta['Shape']
    count = merged_locations_df[['Latitude', 'Longitude']].apply(lambda x: shapeContainsRow(tract_shape, x), axis=1).sum()
    tracts_df.set_value(index=i, col='Total Businesses', value=count)
    tracts_df.set_value(index=i, col='Population', value=tract['Population'] * tract_shape_meta['Fraction'])

# Drop tracts outside Gainesvlle
tracts_df.drop(axis='index', labels=outside_list, inplace=True)

total = tracts_df['Total Businesses'].sum()
print total, '/', len(merged_locations_df), 'businesses matched with tracts'

4937 / 5528 businesses matched with tracts


In [20]:
# Count number of businesses within Gainesville city limits
count = merged_locations_df[['Latitude', 'Longitude']].apply(lambda x: shapeContainsRow(gainesville_shape, x), axis=1).sum()

print 'Number of businesses within Gainesille city limits', count
print count - total, 'businesses within city limits could not be matched with a tract'

Number of businesses within Gainesille city limits 4951
14 businesses within city limits could not be matched with a tract


In [21]:
dglim.saveData(tracts_df, 'Tracts Dataset')

tracts_df[:5]

,Tract,Population,Population Female,Population Over 18,Population Female Over 18,GEOID,Population Working,Average Income,Total Businesses,Somewhat Successful,Somewhat Unsuccessful,Very Successful,Very Unsuccessful,Total Responses
Tract,,,,,,,,,,,,,,
1000,1000,6075,2766,5490,2452,12001001000,2397.0,407098.0,171,7.0,1.0,9.0,1.0,18.0
1100,1100,7220,3893,5400,2895,12001001100,2974.0,375410.0,393,14.0,2.0,16.0,0.0,32.0
1201,1201,3226,1795,2486,1384,12001001201,1264.0,90937.0,109,6.0,0.0,3.0,0.0,9.0
1202,1202,6531,3389,5064,2575,12001001202,2972.0,350261.0,116,4.0,1.0,6.0,0.0,11.0
1203,1203,3689,1964,2801,1598,12001001203,1531.0,118136.0,75,4.0,1.0,2.0,0.0,7.0


In [29]:
tracts_df

,Tract,Population,Population Female,Population Over 18,Population Female Over 18,GEOID,Population Working,Average Income,Total Businesses,Somewhat Successful,Somewhat Unsuccessful,Very Successful,Very Unsuccessful,Total Responses
Tract,,,,,,,,,,,,,,
1000,1000,6075,2766,5490,2452,12001001000,2397.0,407098.0,171,7.0,1.0,9.0,1.0,18.0
1100,1100,7220,3893,5400,2895,12001001100,2974.0,375410.0,393,14.0,2.0,16.0,0.0,32.0
1201,1201,3226,1795,2486,1384,12001001201,1264.0,90937.0,109,6.0,0.0,3.0,0.0,9.0
1202,1202,6531,3389,5064,2575,12001001202,2972.0,350261.0,116,4.0,1.0,6.0,0.0,11.0
1203,1203,3689,1964,2801,1598,12001001203,1531.0,118136.0,75,4.0,1.0,2.0,0.0,7.0
1400,1400,664,1583,3513,1261,12001001400,876.0,173864.0,5,0.0,0.0,0.0,0.0,0.0
1514,1514,1427,772,1087,540,12001001514,691.0,76446.0,15,4.0,0.0,3.0,0.0,7.0
1515,1515,4872,2578,4175,2216,12001001515,2169.0,95119.0,69,3.0,0.0,2.0,1.0,6.0
1516,1516,1804,962,1465,794,12001001516,944.0,72423.0,5,0.0,0.0,1.0,0.0,1.0
